# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [2]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
# from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict, create_text_dict_from_folder

In [15]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 300)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [16]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

# Create text_dict

In [3]:
folder_path = '../text/2023-04-14'
encoding='ISO-8859-1'
subset=None

text_dict = create_text_dict_from_folder(folder_path, encoding='ISO-8859-1', subset=subset)
text_dict

{1: 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during middl

## Create relevance_prompts_df

In [20]:
from itertools import product
experiment_num = 1

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

summarize_task = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]

simple_simplify_task = [
    # "Use terms a 12-year-old can understand.",
    "Assume your audience has no science background.",
    "Use a fun tone."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

user_simplify_task = [
    "Use language appropriate for",
    # "Use terms a 12-year-old can understand.",
]

simplify_audience = [
    # "",
    "a lay audience",
    "people without a science background",
]

user_relevance_task = [
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
]

relevance_audience = [
    # "lay audience",
    # "",
    "seniors",
    "people who enjoy sports"
]



,task part 3,audience,relevance
0,Add 1-2 sentences to make this relevant for,seniors,Add 1-2 sentences to make this relevant for seniors
1,Add 1-2 sentences to make this relevant for,people who enjoy sports,Add 1-2 sentences to make this relevant for people who enjoy sports


# *End of Page*